In [3]:
# import project directory helper
import os, sys
ROOT_DIR = os.path.abspath('/home/hduser/document/jupyter/FYP/') 
sys.path.insert(0, ROOT_DIR)

In [4]:
# import util packages
from IPython.display import display
import ipywidgets as w
import pandas as pd
import re

In [5]:
# import pyspark packages
# set the kafka dependencies before create spark context or session
# import os
# os.environ[
#     'PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4,org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.4 pyspark-shell'
from pyspark.sql import SparkSession, functions, types
from pyspark.sql.types import *

In [6]:
spark = SparkSession.builder.appName('attraction').getOrCreate()
spark

In [7]:
# define path
ds_dir = ROOT_DIR + '/crawler/datasets/tripadvisor_dataset/attractions/'
spark_warehouse_dir = ROOT_DIR + '/crawler/datasets/tripadvisor_dataset/attractions/spark-warehouse/'

In [9]:
# read spark dataframe from parquet
final_attr_spark_df = spark.read.parquet(spark_warehouse_dir + 'etl/attractions')

In [10]:
final_attr_spark_df.createOrReplaceTempView('final_attr_spark_df')

In [11]:
attr_price_spark_df = spark.sql(
    "SELECT MIN(price) as min_price, MAX(price) as max_price FROM final_attr_spark_df"
)

In [12]:
attr_price_df = attr_price_spark_df.toPandas()

In [13]:
attr_category_df = final_attr_spark_df.select(final_attr_spark_df.category).distinct().toPandas()

In [14]:
attr_city_df = final_attr_spark_df.select(final_attr_spark_df.city).distinct().toPandas()

# User details and Preferences

In [15]:
w_uname = w.Text(placeholder='Liho', description="User Name")
w_destination = w.Dropdown(options=attr_city_df.city.tolist(),
                           value='Kuala Lumpur',
                           placeholder='Kuala Lumpur',
                           description="Destination")
w_budget = w.IntRangeSlider(
    min=attr_price_df.min_price[0],
    max=attr_price_df.max_price[0],
    step=10,
    value=[attr_price_df.min_price[0], attr_price_df.max_price[0]],
    description="Budget")
v1 = w.VBox([w_uname, w_destination])

w_start_date = w.DatePicker(description='Start Date', disabled=False)
w_end_date = w.DatePicker(description='End Date', disabled=False)
v2 = w.VBox([w_start_date, w_end_date])

out = w.HBox([v1, v2, w_budget])
display(out)

In [16]:
attr_category_list = attr_category_df.category.tolist()

In [17]:
attr_category_title_list = [' '.join(i.split('_')).title() for i in attr_category_list]

In [18]:
w_cat_rating = dict()


def btn_cat_rating(btn):
    cat = '_'.join(btn.description.split(' ')).lower()

    if cat in w_cat_rating:
        return
    print(btn.description)
    slider = w.IntSlider(min=0, max=5, step=1, description='Rate')
    display(slider)
    w_cat_rating[cat] = slider
    if (len(w_cat_rating) < 5):
        print("Rate {x} more!\n".format(x=5 - len(w_cat_rating)))

In [19]:
button_layout = w.Layout(width='100%', height='50px', border="1px solid black")
button_items = [
    w.Button(description=cat, layout=button_layout)
    for cat in attr_category_title_list
]

on_click = [item.on_click(btn_cat_rating) for item in button_items]

buttons = w.GridBox(button_items,
                    layout=w.Layout(grid_template_columns="repeat(4, 24%)"))

print("Select and rate at least 5 categories and rate them: ")
display(buttons)

Select and rate at least 5 categories and rate them: 


GridBox(children=(Button(description='Air Helicopter Balloon Tours', layout=Layout(border='1px solid black', h…

Food Wine Nightlife


IntSlider(value=0, description='Rate', max=5)

Rate 4 more!

Local Experiences


...

IntSlider(value=0, description='Rate', max=5)

Rate 2 more!

Family Friendly


IntSlider(value=0, description='Rate', max=5)

Rate 1 more!

Spa Tours


IntSlider(value=0, description='Rate', max=5)

In [20]:
uname = re.sub(' ', '_', w_uname.value.lower())
destination = w_destination.value

(budget_low, budget_high) = tuple([float(i) for i in w_budget.value])

start_date = w_start_date.value
end_date = w_end_date.value
    
cat_rating = {key: float(value.value) for key, value in w_cat_rating.items()}

# Model recommendation and filtering

In [21]:
from attraction_recc import *

Instructions for updating:
non-resource variables are not supported in the long term


[nltk_data] Downloading package wordnet to /home/hduser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
hyperparameter = {
    'rows': 5000,
    'epochs': 20,
    'batch_size': 8,
    'alpha': 0.01,
    'H': 128
}
# hyperparameter = {
#     'rows': 40000,
#     'epochs': 50,
#     'batch_size': 16,
#     'alpha': 0.01,
#     'H': 128
# }

In [23]:
filename, user, rbm_att = get_recc(spark, cat_rating, hyperparameter)

Reading the data from /home/hduser/document/jupyter/FYP/crawler/datasets/tripadvisor_dataset/attractions/spark-warehouse/etl/attractions
Reading the data from /home/hduser/document/jupyter/FYP/crawler/datasets/tripadvisor_dataset/attractions/spark-warehouse/etl/attraction_reviews
Extracting 5000 rows from ratings
Preprocessing the dataset
Similar User: 2505
Model restored


,activityId,Recommendation Score
0,11449623,0.000132
1,11449646,0.000127
2,11449648,0.000105
3,11449649,0.000118
4,11449657,0.000092


In [24]:
with_url = filter_df(spark, filename, user, budget_low, budget_high,
                     destination, final_attr_spark_df.toPandas())

In [25]:
final = get_recc_final(with_url, start_date, end_date)


Item no.: 1 --> Item name = Private Kuala Lumpur Grand Tour with Lunch
Evaluating...
Getting URLs without downloading images...
Image URL: https://media.tacdn.com/media/attractions-splice-spp-674x446/07/c7/24/d2.jpg
Printed url without downloading

Errors: 0


Item no.: 1 --> Item name = Hot Spring and Templers Park Waterfall Tour from Kuala Lumpur - 1 Day
Evaluating...
Getting URLs without downloading images...
Image URL: https://media.tacdn.com/media/attractions-splice-spp-674x446/09/96/5e/ee.jpg
Printed url without downloading

Errors: 0


Item no.: 1 --> Item name = Day Tour Hill Resort Cameron Highlands With Batu Caves 
Evaluating...
Getting URLs without downloading images...

Errors: 0


Item no.: 1 --> Item name = Day Tour Hill Resort Cameron Highlands With Batu Caves 
Evaluating...
Getting URLs without downloading images...
Image URL: https://media.tacdn.com/media/attractions-splice-spp-674x446/06/e8/31/92.jpg
Printed url without downloading

Errors: 0


Item no.: 1 --> Item n

# Final Recommendation Results

In [26]:
days = (end_date - start_date).days + 1
days

5

In [27]:
time = ['MORNING', 'EVENING']
fields = ['NAME', 'CATEGORY', 'LOCATION', 'PRICE', 'RATING']
recommendations = ['Recommendation 1:', 'Recommendation 2:']

In [28]:
from ipywidgets import HBox, VBox, widgets

In [29]:
box_layout = w.Layout(
    justify_content='space-between',
    display='flex',
    flex_flow='row',
    align_items='stretch',
)
column_layout = w.Layout(
    justify_content='space-between',
    width='75%',
    display='flex',
    flex_flow='column',
)
tab = []

In [30]:
for i in range(days):
    name = [
        re.sub('_', ' ', i).capitalize()
        for i in final['name'][i*4:(i+1)*4]
    ]
    category = [
        re.sub('_', ' ', i).capitalize()
        for i in final['category'][i*4:(i+1)*4]
    ]
    location = [
        "(" + str(i[0]) + "," + str(i[1]) + ")"
        for i in final['location'][i*4:(i+1)*4]
    ]
    price = [str(i) for i in final['price']][i*4:(i+1)*4]
    rating = [str(i) for i in final['rating']][i*4:(i+1)*4]

    images = final['image'][i*4:(i+1)*4]
    tab.append(
        VBox(children=[
            HBox(children=[
                VBox(children=[
                    widgets.HTML(value=f"<b><font color='orange'>{time[0]}</b>"),
                    widgets.HTML(
                        value=f"<b><font color='purple'>{recommendations[0]}</b>"),
                    widgets.HTML(value="<img src='{}'>".format(images[0])),
                    widgets.HTML(description=fields[0],
                                 value=f"<b><font color='black'>{name[0]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[1],
                                 value=f"<b><font color='black'>{category[0]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[2],
                                 value=f"<b><font color='black'>{location[0]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[3],
                                 value=f"<b><font color='black'>{price[0]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[4],
                                 value=f"<b><font color='black'>{rating[0]}</b>",
                                 disabled=True)
                ],
                     layout=column_layout),
                VBox(children=[
                    widgets.HTML(value=f"<b><font color='orange'>{time[1]}</b>"),
                    widgets.HTML(
                        value=f"<b><font color='purple'>{recommendations[0]}</b>"),
                    widgets.HTML(value="<img src='{}'>".format(images[2])),
                    widgets.HTML(description=fields[0],
                                 value=f"<b><font color='black'>{name[2]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[1],
                                 value=f"<b><font color='black'>{category[2]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[2],
                                 value=f"<b><font color='black'>{location[2]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[3],
                                 value=f"<b><font color='black'>{price[2]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[4],
                                 value=f"<b><font color='black'>{rating[2]}</b>",
                                 disabled=True)
                ],
                     layout=column_layout)
            ],
                 layout=box_layout),
            HBox(children=[
                VBox(children=[
                    widgets.HTML(
                        value=f"<b><font color='purple'>{recommendations[1]}</b>"),
                    widgets.HTML(value="<img src='{}'>".format(images[1])),
                    widgets.HTML(description=fields[0],
                                 value=f"<b><font color='black'>{name[1]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[1],
                                 value=f"<b><font color='black'>{category[1]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[2],
                                 value=f"<b><font color='black'>{location[1]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[3],
                                 value=f"<b><font color='black'>{price[1]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[4],
                                 value=f"<b><font color='black'>{rating[1]}</b>",
                                 disabled=True)
                ],
                     layout=column_layout),
                VBox(children=[
                    widgets.HTML(
                        value=f"<b><font color='purple'>{recommendations[1]}</b>"),
                    widgets.HTML(value="<img src='{}'>".format(images[3])),
                    widgets.HTML(description=fields[0],
                                 value=f"<b><font color='black'>{name[3]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[1],
                                 value=f"<b><font color='black'>{category[3]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[2],
                                 value=f"<b><font color='black'>{location[3]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[3],
                                 value=f"<b><font color='black'>{price[3]}</b>",
                                 disabled=True),
                    widgets.HTML(description=fields[4],
                                 value=f"<b><font color='black'>{rating[3]}</b>",
                                 disabled=True)
                ],
                     layout=column_layout),
            ],
                 layout=box_layout)
        ]))

In [31]:
tab_recc = widgets.Tab(children=tab)
for i in range(len(tab_recc.children)):
    tab_recc.set_title(i, str('Day ' + str(i + 1)))

In [32]:
display(tab_recc)